# test de la sincro son, image


dans ce fichier on test la création d'un estimateur tres simple image/son

La personne qui parle est celle la plus proche de la source de son.

In [1]:
import inspect

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold

from metric.compute_metric import compute_clear
from utils import make_pred, save_data, iter_frame_sync
from metric.read_file import read_file
from utils import Point, mult_load
import numpy as np


In [ ]:
from utils import iter_frame, Corp
from utils.identity_assignation import identity_assignation


class InertieClasificator(BaseEstimator, ClassifierMixin):
    def __init__(self, coef_ine_hd = 1, coef_ine_bd =1, coef_dist_hd = 1, coef_dist_bd = 1, 
                 coef_size_hd = 1, coef_size_bd = 1, coef_pt_cache = 10, mem_length= 20, coef_col = 1):
        
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")

        for arg, val in values.items():
            setattr(self, arg, val)
    
    def fit(self, X, y=None):
        # on n'apprend rien pour l'instant
        pass
    
    def mk_dist(self):
        def fdist(id, corp):
            var = id.comp(corp)
            dist = var[0] * self.coef_ine_hd
            dist += var[1] * self.coef_ine_bd
            dist += var[2] * self.coef_col
            dist += var[3] * self.coef_dist_hd
            dist += var[4] * self.coef_dist_bd
            dist += var[5] * self.coef_pt_cache
            return dist
        return fdist
               
    
    def predict(self, X, y=None):
        # prediction
        
        def func (data):
            data_pos, data_son = data
            identites = dict()
            for frame_pos, frame_son in iter_frame_sync(data_pos,data_son):
                
                son = frame_son[0]
                identites = identity_assignation(frame_pos, identites, None, self.mk_dist())
    
                for i, l in enumerate(frame_pos):
                    cp = Corp(l)
                    parle = 0
                    # print(str(Point(son[2], son[1])) + "vs " + str(cp.hd_bary))
                    if cp.hd_bary.dist(Point(son[2], son[1])) < 30:
                        # print("Match !! ")
                        parle = 1
                    yield (identites[cp], parle)  
        
        return [make_pred(vid[0], func, data_son=vid[1]) for vid in X]
    
    def score(self, X, y, sample_weight=None):
        pred = self.predict(X)
        res = [0, 0, 0, 0]
        for vid, raw in zip(pred, y):
            c = compute_clear(raw, vid, 1)
            c = list(c)
            res = [a+b for a,b in zip(res, c)]
        # print(res/len(y))
        return [x/len(y) for x in res]
           
        

In [5]:
data_ref, data = mult_load(["data/Scenario03-02/", "data/Scenario01-01/", "data/Scenario02-03/", "data/Scenario02-01/",
                            "data/Scenario02-02/", "data/Scenario03-03/", "data/Scenario03-02/", "data/Scenario03-01/",
                            "data/Scenario03-05/"], son=True)

In [10]:
est = InertieClasificator()
print(est.score(data, data_ref))


[0.97997403766573854, 0.0, 0.65668050328408545, 0.63922655684137064]


parole a 0 : 0.977; 0 , 0.54, 0.53


In [ ]:
PATH = "data/Scenario02-03/"
data_ref = read_file(PATH+"groundTruth.txt")
data = np.loadtxt(PATH+"augmented_data.txt")
data_son = np.loadtxt(PATH+"ssl_detections.txt")
est = InertieClasificator()
res = est.predict([(data, data_son)], [data_ref])
print(est.score([(data, data_son)], [data_ref]))

In [4]:
res

[array([[  35.,    1.,    1., ...,   58.,   -1.,   -1.],
        [  36.,    1.,    1., ...,   56.,   -1.,   -1.],
        [  37.,    1.,    1., ...,   58.,   -1.,   -1.],
        ..., 
        [ 424.,    2.,    0., ...,   88.,   -1.,   -1.],
        [ 425.,    1.,    0., ...,   -1.,   52.,  104.],
        [ 425.,    2.,    0., ...,   -1.,   -1.,   -1.]])]

In [5]:
save_data(res[0], PATH +  "res_tmp")